<a href="https://colab.research.google.com/github/krizz18bugger/Brain-Tumor-Classification/blob/main/Brain_abnormality_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img
from tensorflow.keras.applications import DenseNet121
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense,GlobalAveragePooling2D,Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, fbeta_score, roc_auc_score,classification_report
import numpy as np
train_datagen=ImageDataGenerator(
    rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255,validation_split=0.5)
train_set =train_datagen.flow_from_directory(
    '/content/drive/MyDrive/Brain Tumor Dataset/Training',  # Folder containing training images
    target_size=(224, 224),
    batch_size=64,
    shuffle=True,
    class_mode='categorical'
)
valid_set=test_datagen.flow_from_directory(
    '/content/drive/MyDrive/Brain Tumor Dataset/Testing',  # Folder containing validation images
    target_size=(224, 224),
    subset="validation",
    batch_size=32,
    shuffle=False,
    class_mode='categorical'
)
test_set= test_datagen.flow_from_directory(
    '/content/drive/MyDrive/Brain Tumor Dataset/Testing',  # Folder containing test images
    target_size=(224, 224),
    subset="training",
    batch_size=32,
    shuffle=False,
    class_mode='categorical'
)

class_names = sorted(train_set.class_indices.keys())
print(class_names)
AUTOTUNE=tf.data.AUTOTUNE

Mounted at /content/drive
Found 5713 images belonging to 4 classes.
Found 656 images belonging to 4 classes.
Found 657 images belonging to 4 classes.
['glioma', 'meningioma', 'notumor', 'pituitary']


In [2]:
base_model=DenseNet121(weights='imagenet',include_top=False,input_shape=(224,224,3))
base_model.trainable=False
x=base_model.output
x=GlobalAveragePooling2D()(x)
x=Dropout(0.55)(x)
x=Dense(128,activation='relu')(x)
x=Dropout(0.55)(x)
out=Dense(4,activation='sigmoid')(x)
early_stop = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
model=Model(inputs=base_model.input,outputs=out)
model.compile(optimizer=Adam(learning_rate=0.001),loss='binary_crossentropy',metrics=['accuracy'])
mod_fit=model.fit(train_set,epochs=80,validation_data=valid_set,callbacks=[early_stop])
test_loss, test_acc = model.evaluate(test_set)

29084464/29084464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/80
90/90 ━━━━━━━━━━━━━━━━━━━━ 1546s 17s/step - accuracy: 0.4218 - loss: 0.6948 - val_accuracy: 0.7348 - val_loss: 0.3462
Epoch 2/80
90/90 ━━━━━━━━━━━━━━━━━━━━ 32s 354ms/step - accuracy: 0.6648 - loss: 0.3726 - val_accuracy: 0.7576 - val_loss: 0.2914
Epoch 3/80
90/90 ━━━━━━━━━━━━━━━━━━━━ 32s 358ms/step - accuracy: 0.7019 - loss: 0.3322 - val_accuracy: 0.7744 - val_loss: 0.2739
Epoch 4/80
90/90 ━━━━━━━━━━━━━━━━━━━━ 32s 355ms/step - accuracy: 0.7519 - loss: 0.3004 - val_accuracy: 0.7622 - val_loss: 0.2614
Epoch 5/80
90/90 ━━━━━━━━━━━━━━━━━━━━ 33s 362ms/step - accuracy: 0.7778 - loss: 0.2682 - val_accuracy: 0.8018 - val_loss: 0.2469
Epoch 6/80
90/90 ━━━━━━━━━━━━━━━━━━━━ 34s 379ms/step - accuracy: 0.7735 - loss: 0.2671 - val_accuracy: 0.7790 - val_loss: 0.2453
Epoch 7/80
90/90 ━━━━━━━━━━━━━━━━━━━━ 33s 360ms/step - accuracy: 0.7842 - loss: 0.2546 - val_accuracy: 0.7973 - val_loss: 0.2456
Epoch 8/80
90/90 ━━━━━━━━━━━━━━━━━━━━ 32s 350ms/step - accuracy: 0.7857 - loss: 0.2549 - val_accu

In [6]:
# Getting predicted probabilities
y_pred_prob = model.predict(test_set)
y_pred_prob_normalized = tf.nn.softmax(y_pred_prob, axis=1).numpy()

# Converting probabilities to single integer labels by taking the argmax
y_pred = np.argmax(y_pred_prob, axis=1)
y_true = test_set.classes

print('Test accuracy:', test_acc)
print('Test_loss:',test_loss)

# Calculating metrics
accuracy = accuracy_score(y_true, y_pred)
precision = precision_score(y_true, y_pred, average='macro')
recall = recall_score(y_true, y_pred, average='macro')
f1 = f1_score(y_true, y_pred, average='macro')
f2 = fbeta_score(y_true, y_pred, beta=2, average='macro')
auc_roc = roc_auc_score(y_true, y_pred_prob_normalized, multi_class='ovr', average='macro')

#Results
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-score: {f1:.4f}")
print(f"F2-score: {f2:.4f}")
print(f"AUC-ROC: {auc_roc:.4f}")

# Classification Report
report = classification_report(y_true, y_pred, target_names=class_names)
print(report)

model.save('/content/drive/MyDrive/Brain_Tumor/brainTumorClassification_model.h5')

21/21 ━━━━━━━━━━━━━━━━━━━━ 3s 140ms/step


Test accuracy: 0.8767123222351074
Test_loss: 0.13385428488254547
Accuracy: 0.8767
Precision: 0.8895
Recall: 0.8660
F1-score: 0.8624
F2-score: 0.8617
AUC-ROC: 0.9828
              precision    recall  f1-score   support

      glioma       0.99      0.60      0.75       150
  meningioma       0.70      0.88      0.78       153
     notumor       1.00      1.00      1.00       204
   pituitary       0.87      0.99      0.93       150

    accuracy                           0.88       657
   macro avg       0.89      0.87      0.86       657
weighted avg       0.90      0.88      0.87       657

